In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pandas_datareader as dr
import datetime

In [2]:
#Assumptions

company_ticker = 'JBHT'

# Constant Expected Growth Rate
long_term_growth = 0.02

# Market Risk Premium
market_risk_premium = 0.0523

# Risk-free Rate
timespan = 100
current_date = datetime.date.today()
past_date = current_date-datetime.timedelta(days=timespan)
risk_free_rate_df = dr.DataReader('^TNX', 'yahoo', past_date, current_date) 
risk_free_rate = (risk_free_rate_df.iloc[len(risk_free_rate_df)-1,5])/100
print(risk_free_rate)

debt_spread = 0.020




0.041560001373291015


In [3]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/cash-flow"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

cash_flow_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Cash Flow Statement DataFrame
cash_flow_df = pd.DataFrame.from_dict(cash_flow_response.json())
cash_flow_df = cash_flow_df.drop('ttm', axis = 1)
cash_flow_df

,12/31/2018,12/31/2019,12/31/2020,12/31/2021
Capital Expenditure,-995650,-854115,-738545,-947563
End Cash Position,7600,35000,313302,355549
Financing Cash Flow,-208080,-266985,-231600,-304633
Free Cash Flow,92191,244232,384314,276335
Income Tax Paid Supplemental Data,83822,71681,95454,203740
Interest Paid Supplemental Data,39901,46721,48351,47016
Investing Cash Flow,-886773,-803962,-612957,-877018
Issuance of Debt,3204715,2291014,222124,0
Operating Cash Flow,1087841,1098347,1122859,1223898
Repayment of Debt,-3137900,-2154000,-220100,0


In [4]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/income-statement"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

income_statement_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Income Statement DataFrame
income_statement_df = pd.DataFrame.from_dict(income_statement_response.json())
income_statement_df = income_statement_df.drop('ttm', axis = 1)
income_statement_df

,12/31/2018,12/31/2019,12/31/2020,12/31/2021
Basic Average Shares,108711,106213,105654,105094
Cost of Revenue,7255657,7659003,8186697,10298483
Diluted Average Shares,110428,108599,106766,106593
Diluted NI Available to Com Stockholders,489585,516320,506035,760806
EBIT,681245,735579,713605,1046023
Gross Profit,1359217,1506255,1449876,1869819
Interest Expense,40427,54684,47580,46251
Interest Income,224,1754,486,493
Net Income Common Stockholders,489585,516320,506035,760806
Net Income from Continuing & Discontinued Operation,489585,516320,506035,760806


In [5]:
# FCFF Calculation using Cash Flow Statement and Income Statement Inputs
free_cash_flow_firm = (cash_flow_df.loc['Free Cash Flow'].astype(int) \
                    + (income_statement_df.loc['Interest Expense'].astype(int) \
                       * (1 - income_statement_df.loc['Tax Provision'].astype(int) \
                          / income_statement_df.loc['Pretax Income'].astype(int)))).astype(int)

# Change Series to a Pandas Dataframe
free_cash_flow_firm_df = free_cash_flow_firm.to_frame().transpose()
free_cash_flow_firm_df

,12/31/2018,12/31/2019,12/31/2020,12/31/2021
0,123077,285698,420464,311531


In [36]:
# CAGR of FCFF
latest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,len(free_cash_flow_firm_df.columns)-1])
earliest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,1])
free_cash_flow_firm_CAGR = (latest_free_cash_flow_firm/earliest_free_cash_flow_firm)\
                            **(float(1/(len(free_cash_flow_firm_df.columns)-2)))-1
free_cash_flow_firm_CAGR = .3

In [37]:
# Forecasted FCFF
forecast_free_cash_flow_firm_df = pd.DataFrame(columns=['Year ' + str(i) for i in range(1,7)])
free_cash_flow_firm_forecast_lst = []
for i in range(1,7):
    if i != 6:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)**i
    else:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)\
                                        **(i-1)*(1+long_term_growth)
    free_cash_flow_firm_forecast_lst.append(int(free_cash_flow_firm_forecast))
forecast_free_cash_flow_firm_df.loc[0] = free_cash_flow_firm_forecast_lst
forecast_free_cash_flow_firm_df

,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6
0,404990,526487,684433,889763,1156692,1179826


In [8]:
url = "https://stock-analysis.p.rapidapi.com/api/v1/resources/key-stats"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "stock-analysis.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

key_stats_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Key Statistics DataFrame
key_stats_df = pd.DataFrame.from_dict(key_stats_response.json())
key_stats_df = key_stats_df.transpose()
key_stats_df

,Value
% held by insiders,20.91%
% held by institutions,81.13%
200-day moving average,177.81
5-year average dividend yield,0.83
50-day moving average,167.83
52-week change,-11.03%
52-week high,218.18
52-week low,153.92
Avg vol (10-day),629.77k
Avg vol (3-month),764.38k


In [9]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/balance-sheet"
querystring = {"ticker":company_ticker}
headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }
balance_sheet_response = requests.request("GET", url, headers=headers, params=querystring)
# Create Balance Sheet DataFrame
balance_sheet_df = pd.DataFrame.from_dict(balance_sheet_response.json())
balance_sheet_df

,12/31/2018,12/31/2019,12/31/2020,12/31/2021
Common Stock Equity,2101384,2267029,2600138,3117816
Invested Capital,3250488,3562769,3905562,4419045
Net Debt,1141504,1260740,992122,945680
Net Tangible Assets,1996197,2064203,2387971,2926695
Ordinary Shares Number,108711,106213,105654,105094
Share Issued,167099,167099,167099,167099
Tangible Book Value,1996197,2064203,2387971,2926695
Total Assets,5091647,5470854,5928348,6794348
Total Capitalization,2999782,3562769,3905562,4063073
Total Debt,1149104,1295740,1305424,1301229


In [10]:
# Stock's Beta
equity_beta = float(key_stats_df.loc[r'Beta (5Y monthly)'])
equity_beta

1.13

In [11]:
# Required Return on Equity
equity_return = risk_free_rate + (equity_beta*market_risk_premium)
equity_return

0.10065900137329101

In [12]:
# Interest Expense
interest_expense = income_statement_df.loc['Interest Expense']
interest_expense_df = interest_expense.to_frame().transpose()
interest_expense_str = interest_expense_df.values[0][-1:]
interest_expense_int = int(interest_expense_str)
# Total Debt
total_debt = balance_sheet_df.loc['Total Debt']
total_debt_df = total_debt.to_frame().transpose()
total_debt_str = total_debt_df.values[0][-1:]
total_debt_int = int(total_debt_str)
# Required Return on Debt
debt_return = risk_free_rate + debt_spread 
debt_return

0.06156000137329101

In [13]:
# Effective Tax Rate
effective_tax_rate = income_statement_df.loc['Tax Provision'].astype(int) / income_statement_df.loc['Pretax Income'].astype(int)
avg_effective_tax_rate = sum(effective_tax_rate) / len(effective_tax_rate)
avg_effective_tax_rate

0.23923514091557435

In [28]:
# Market Value of Equity
market_cap_str = key_stats_df.loc[r'Market cap (intra-day)'][0]
market_cap_lst = market_cap_str.split('.')

if market_cap_str[len(market_cap_str)-1] == 'T':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('T',(9-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'B':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('B',(6-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'M':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('M',(3-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))

print("Market Cap = " +'${:,.0f}'.format(market_cap_int))

Market Cap = $17,940,000


In [29]:
# Market Value of Debt
net_debt = balance_sheet_df.loc['Net Debt']
net_debt_df = net_debt.to_frame().transpose()
net_debt_str = net_debt_df.values[0][-1:]
net_debt_int = int(net_debt_str)

print("Net Debt = " +'${:,.0f}'.format(net_debt_int))

Net Debt = $945,680


In [30]:
# Company Value
company_value = market_cap_int + net_debt_int
print("Company Value = " +'${:,.0f}'.format(company_value))

Company Value = $18,885,680


In [17]:
WACC = ((market_cap_int/company_value) * equity_return) \
        + ((net_debt_int/company_value) * (debt_return * (1-avg_effective_tax_rate)))

print("WACC = "+ '{0:,.2f}%'.format(WACC*100))

WACC = 9.80%


In [38]:
# Equity Value Calculation
discounted_FCFF_lst = []
for year in range(0,5):
    discounted_FCFF = forecast_free_cash_flow_firm_df.iloc[0,year]/(1+WACC)**(year+1)
    discounted_FCFF_lst.append(int(discounted_FCFF))
    
terminal_value = forecast_free_cash_flow_firm_df.iloc[0,5]/(WACC-long_term_growth)
print("Terminal Value = " +'${:,.0f}'.format(terminal_value))

PV_terminal_value = int(terminal_value/(1+WACC)**5)
print("PV of terminal Value = " +'${:,.0f}'.format(PV_terminal_value))

enterprise_value = sum(discounted_FCFF_lst)+PV_terminal_value
print("Enterprise Value = " +'${:,.0f}'.format(enterprise_value))


equity_value = enterprise_value-net_debt_int
print("Equity Value = " +'${:,.0f}'.format(equity_value))

Terminal Value = $15,133,016
PV of terminal Value = $9,483,869
Enterprise Value = $12,143,682
Equity Value = $11,198,002


In [26]:
# Total Shares Outstanding
shares_outstanding_str = key_stats_df.loc[r'Shares outstanding '][0]
shares_outstanding_lst = shares_outstanding_str.split('.')


if shares_outstanding_str[len(shares_outstanding_str)-1] == 'T':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('T',(9-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'B':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('B',(6-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'M':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('M',(3-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))

print(shares_outstanding_str)
shares_outstanding_int

103.54M


103540

In [39]:
# Two-stage FCFF Valuation Model Stock Price Estimate
stock_price = equity_value / shares_outstanding_int
stock_price = '${:,.2f}'.format(stock_price)
print("Model Stock Price = %s"%(stock_price))
# Actual Stock Price
actual_stock_price = market_cap_int / shares_outstanding_int
actual_stock_price = '${:,.2f}'.format(actual_stock_price)
print("Actual Stock Price = %s"%(actual_stock_price))

Model Stock Price = $108.15
Actual Stock Price = $173.27
